In [3]:
from PredictionModel.convertDcmtoJpg import convertDcm2Jpg

dcmpath = "patient-data/p001/p001_2.dcm"
targetpath = 'patient-data/p001/p001_2.jpg'

convertDcm2Jpg(dcmpath, targetpath)

In [1]:
#!python PredictionModel/runPredModel.py patient-data/p001/p001.jpg
from PredictionModel.runPredModel import scanallxrays
import os
import pprint

pdir = 'patient-data'
for p in os.listdir(pdir):
    patient_dir = os.path.join(pdir, p)
    print(f'patient {p}: {patient_dir}')
    pprint.pprint(scanallxrays(patient_dir))

patient p001: patient-data\p001
{'Atelectasis': 0.21498525887727737,
 'Cardiomegaly': 0.14202095940709114,
 'Consolidation': 0.16093560867011547,
 'Edema': 0.19310505222529173,
 'Effusion': 0.07321334350854158,
 'Emphysema': 0.5159958302974701,
 'Enlarged Cardiomediastinum': 0.3417065441608429,
 'Fibrosis': 0.07502555847167969,
 'Fracture': 0.41019532084465027,
 'Hernia': 0.016630699392408133,
 'Infiltration': 0.37858523428440094,
 'Lung Lesion': 0.23170176148414612,
 'Lung Opacity': 0.4581741467118263,
 'Mass': 0.18438709899783134,
 'Nodule': 0.5161708295345306,
 'Pleural_Thickening': 0.3676226958632469,
 'Pneumonia': 0.43097905814647675,
 'Pneumothorax': 0.17224140092730522}
patient p002: patient-data\p002
{'Atelectasis': 0.17979608476161957,
 'Cardiomegaly': 0.17245332896709442,
 'Consolidation': 0.12583898566663265,
 'Edema': 0.22289014561101794,
 'Effusion': 0.08387733437120914,
 'Emphysema': 0.5172516703605652,
 'Enlarged Cardiomediastinum': 0.3341105952858925,
 'Fibrosis': 0.084

In [24]:
import sys
import os
sys.path.insert(0,"..")
import numpy as np
import cv2
import skimage, skimage.io
import pprint

import torch
import torchvision, torchvision.transforms
import torchxrayvision as xrv
import pydicom
from PredictionModel.convertDcmtoJpg import lin_stretch_img
from PredictionModel.runPredModel import getprediction

def datapred(img, weights="densenet121-res224-all"):

    #img = skimage.io.imread(img_path)
    img = xrv.datasets.normalize(img, 255)  

    # Check that images are 2D arrays
    if len(img.shape) > 2:
        img = img[:, :, 0]
    if len(img.shape) < 2:
        print("error, dimension lower than 2 for image")

    # Add color channel
    img = img[None, :, :]

    
    # the models will resize the input to the correct size so this is optional.
    transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop()])

    img = transform(img)
    model = xrv.models.get_model(weights)

    output = {}
    with torch.no_grad():
        img = torch.from_numpy(img).unsqueeze(0)
        preds = model(img).cpu()
        output["preds"] = dict(zip(xrv.datasets.default_pathologies,preds[0].detach().numpy()))

    diseases = ['Atelectasis', 'Pneumonia', 'Cardiomegaly', 'Pleural_Thickening']
    allpreds = output.get("preds")
    predictions = {}
    if allpreds:
        for disease in diseases:
            predictions[disease] = allpreds.get(disease)
    else :
        print("results not found")
        return {}

    #print(predictions)
    #print(allpreds)
    #print(xrv.models.model_urls.keys())
    #pprint.pprint(allpreds)
    
    return allpreds


def convertDcm2Jpg(dcmpath):

    ds = pydicom.dcmread(dcmpath)
    img = ds.pixel_array # get image array

    img = lin_stretch_img(img, 1, 99)
    return img

def main():
    fl = 'patient-data/p001/p001'
    dcmpath = f'{fl}.dcm'
    jpgath = f'{fl}.jpg'

    imgdata = convertDcm2Jpg(dcmpath)
    pdata = datapred(imgdata)
    pimg = getprediction(jpgath)

    res = {}
    for k in pdata.keys():
        res[k] = pdata[k]-pimg[k]

    pprint.pprint(res)

#main()

dcmpath = 'patient-data/p001/p001_2.dcm'
jpgath = 'patient-data/p001/p001_2.png'
imgdata = convertDcm2Jpg(dcmpath)
#cv2.imwrite(jpgath, imgdata)

img1 = xrv.datasets.normalize(imgdata, 255)
#print(type(img1))

img2 = skimage.io.imread(jpgath)
img2 = xrv.datasets.normalize(img2, 255)

print(np.array_equal(img1, img2, equal_nan=True))

#diff = np.sum(np.abs(img1 - img2))
#print(diff)


True
